In [1]:
import operator
import warnings
from typing import *
import traceback

import torch
from dotenv import load_dotenv
from IPython.display import Image
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import END, StateGraph
from langchain_core.messages import AnyMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_openai import ChatOpenAI
from transformers import logging
import matplotlib.pyplot as plt
import numpy as np

from medmax.agent import *
from medmax.tools import *
from medmax.utils import *

warnings.filterwarnings("ignore")
logging.set_verbosity_error()
_ = load_dotenv()

PROMPT_FILE = "medmax/docs/system_prompts.txt"
SCRATCH_DIR = "/scratch/ssd004/scratch/afallah"

"""
1. SFT or example database, prompt enhancing to make tool call better
2. show segmentation result overlay on the image
3. use ppx instead of cm2, support dicom
4. look agent benchmark + build benchmark
"""

'\n1. SFT or example database, prompt enhancing to make tool call better\n2. show segmentation result overlay on the image\n3. use ppx instead of cm2, support dicom\n4. look agent benchmark + build benchmark\n'

In [2]:
# !pip install -U git+https://github.com/huggingface/transformers.git@88d960937c81a32bfb63356a2e8ecf7999619681
# !pip install -U pydicom gdcm  pylibjpeg 

In [47]:
report_tool = ChestXRayReportGeneratorTool()
xray_classification_tool = ChestXRayClassifierTool()
# medical_visual_qa_tool = MedicalVisualQATool()
segmentation_tool = ChestXRaySegmentationTool()
image_visualizer_tool = ImageVisualizerTool()
# grounding_tool = XRayPhraseGroundingTool(cache_dir=SCRATCH_DIR, temp_dir="temp")
# generation_tool = ChestXRayGeneratorTool(model_path=f"{SCRATCH_DIR}/roentgen", temp_dir="temp")
dicom_tool = DicomProcessorTool(temp_dir="temp")

print(f"Type: {type(report_tool)}, Name: {report_tool.name}")
print(f"Type: {type(xray_classification_tool)}, Name: {xray_classification_tool.name}")
# print(f"Type: {type(medical_visual_qa_tool)}, Name: {medical_visual_qa_tool.name}")
print(f"Type: {type(segmentation_tool)}, Name: {segmentation_tool.name}")
print(f"Type: {type(image_visualizer_tool)}, Name: {image_visualizer_tool.name}")
print(f"Type: {type(grounding_tool)}, Name: {grounding_tool.name}")
print(f"Type: {type(generation_tool)}, Name: {generation_tool.name}")
print(f"Type: {type(dicom_tool)}, Name: {dicom_tool.name}")

Type: <class 'medmax.tools.report_generation.ChestXRayReportGeneratorTool'>, Name: chest_xray_report_generator
Type: <class '__main__.ChestXRayClassifierTool'>, Name: chest_xray_classifier
Type: <class 'medmax.tools.segmentation.ChestXRaySegmentationTool'>, Name: chest_xray_segmentation
Type: <class 'medmax.tools.utils.ImageVisualizerTool'>, Name: image_visualizer
Type: <class 'medmax.tools.grounding.XRayPhraseGroundingTool'>, Name: xray_phrase_grounding
Type: <class 'medmax.tools.generation.ChestXRayGeneratorTool'>, Name: chest_xray_generator
Type: <class '__main__.DicomProcessorTool'>, Name: dicom_processor


In [51]:
prompts = load_prompts_from_file(PROMPT_FILE)
prompt = prompts["MEDICAL_ASSISTANT"]

checkpointer = MemorySaver()
model = ChatOpenAI(model="gpt-4o-2024-08-06", temperature=0, top_p=0.95)
agent = Agent(
    model,
    tools=[
        xray_classification_tool,
        #    report_tool,
        #    segmentation_tool,
        image_visualizer_tool,
        #    grounding_tool,
        #    generation_tool,
        #    medical_visual_qa_tool,
        dicom_tool
    ],
    log_tools=True,
    log_dir="logs",
    system_prompt=prompt,
    checkpointer=checkpointer,
)
thread = {"configurable": {"thread_id": "1"}}

# Image(agent.graph.get_graph().draw_png())

In [52]:
# messages = [HumanMessage(content="Does the image `demo/chest/pneumonia4.jpg` show pneumonia?")]
# messages = [HumanMessage(content="Provide a radiology report for the given image `demo/chest/pneumonia2.jpg`.")]
# messages = [HumanMessage(content="Descibe the given image.")]
# messages = [HumanMessage(content="What was the first question I asked?")]
# messages = [HumanMessage(content="What is the probability of pneumonia in the image `demo/chest/normal1.jpg`?")]
# messages = [HumanMessage(content="Does the patient with chest xray given here need to go see doctor? `demo/chest/normal1.jpg`")]
# messages = [HumanMessage(content="What is the size of heart?")]
messages = [HumanMessage(content="Classify disease in the image `demo/chest/LIDC.dcm`")]

In [53]:
for event in agent.workflow.stream({"messages": messages}, thread):
    for v in event.values():
        print(v, '\n')

{'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2uE9XCfS1akrDg88MFkp731X', 'function': {'arguments': '{"dicom_path":"demo/chest/LIDC.dcm"}', 'name': 'dicom_processor'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 554, 'total_tokens': 577, 'completion_tokens_details': {'audio_tokens': 0, 'reasoning_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_7f6be3efb0', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-62ba5d64-954c-46c7-9b37-03ecc216a1a5-0', tool_calls=[{'name': 'dicom_processor', 'args': {'dicom_path': 'demo/chest/LIDC.dcm'}, 'id': 'call_2uE9XCfS1akrDg88MFkp731X', 'type': 'tool_call'}], usage_metadata={'input_tokens': 554, 'output_tokens': 23, 'total_tokens': 577, 'input_token_details': {'audio': 0, 'cache_rea

In [42]:
import gradio as gr
from pathlib import Path
import time
import shutil
from typing import List, Optional, Tuple
from gradio import ChatMessage

class ChatInterface:
    def __init__(self, agent):
        self.agent = agent
        self.upload_dir = Path("temp")
        self.upload_dir.mkdir(exist_ok=True)
        self.current_thread_id = None
        # Separate storage for original and display paths
        self.original_file_path = None  # For LLM (.dcm or other)
        self.display_file_path = None   # For UI (always viewable format)

    def handle_upload(self, file_path: str) -> str:
        """
        Handle new file upload and set appropriate paths
        Returns: display_path for UI
        """
        if not file_path:
            return None
            
        source = Path(file_path)
        timestamp = int(time.time())
        
        # Save original file
        saved_path = self.upload_dir / f"upload_{timestamp}{source.suffix}"
        shutil.copy2(source, saved_path)
        self.original_file_path = str(saved_path)
        
        # Handle DICOM conversion for display only
        if source.suffix.lower() == '.dcm':
            output, _ = dicom_tool._run(str(saved_path))
            self.display_file_path = output['image_path']
        else:
            self.display_file_path = str(saved_path)
        
        return self.display_file_path

    async def process_message(self, 
                            message: str, 
                            display_image: Optional[str], 
                            chat_history: List[ChatMessage]) -> List[ChatMessage]:
        chat_history = chat_history or []
        
        # Use original file path for LLM prompt
        if self.original_file_path:
            message = f"{message} `{self.original_file_path}`"
        
        # Initialize thread if needed
        if not self.current_thread_id:
            self.current_thread_id = str(time.time())
        
        chat_history.append(ChatMessage(role="user", content=message))
        yield chat_history, self.display_file_path
        
        try:
            for event in self.agent.workflow.stream(
                {"messages": [{"role": "user", "content": message}]},
                {"configurable": {"thread_id": self.current_thread_id}}
            ):
                if isinstance(event, dict):
                    if 'process' in event:
                        content = event['process']['messages'][-1].content
                        if content:
                            chat_history.append(ChatMessage(
                                role="assistant",
                                content=content
                            ))
                            yield chat_history, self.display_file_path

                    elif 'execute' in event:
                        for message in event['execute']['messages']:
                            tool_name = message.name
                            tool_result = eval(message.content)[0]
                            
                            # For image_visualizer, use display path
                            if tool_name == "image_visualizer":
                                if self.display_file_path:  # Keep using current display path
                                    message.args['image_path'] = self.display_file_path
                            
                            if tool_result:
                                formatted_result = ' '.join(line.strip() for line in str(tool_result).splitlines()).strip()
                                chat_history.append(ChatMessage(
                                    role="assistant",
                                    content=formatted_result,
                                    metadata={"title": f"🔧 Using tool: {tool_name}"},
                                ))
                                yield chat_history, self.display_file_path
                            
        except Exception as e:
            chat_history.append(ChatMessage(
                role="assistant",
                content=f"❌ Error: {str(e)}",
                metadata={"title": "Error"}
            ))
            yield chat_history, self.display_file_path

def create_demo(agent):
    interface = ChatInterface(agent)
    
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        with gr.Column():
            gr.Markdown("""
            # 🏥 MedMAX
            Multimodal Medical Agent for Chest X-rays
            """)
            
            with gr.Row():
                with gr.Column(scale=3):
                    chatbot = gr.Chatbot(
                        [],
                        height=800,
                        container=True,
                        show_label=True,
                        elem_classes="chat-box",
                        type="messages",
                        label="Agent",
                        avatar_images=(None, "https://em-content.zobj.net/source/twitter/53/robot-face_1f916.png")
                    )
                    with gr.Row():
                        with gr.Column(scale=3):
                            txt = gr.Textbox(
                                show_label=False,
                                placeholder="Ask about the X-ray...",
                                container=False
                            )
                            
                with gr.Column(scale=3):
                    image_display = gr.Image(
                        label="Image",
                        type="filepath",
                        height=700,
                        container=True
                    )
                    with gr.Row():
                        upload_button = gr.UploadButton(
                            "📎 Upload X-Ray",
                            file_types=["image"],
                        )
                        dicom_upload = gr.UploadButton(
                            "📄 Upload DICOM",
                            file_types=["file"],
                        )
                    with gr.Row():
                        clear_btn = gr.Button("Clear Chat")
                        new_thread_btn = gr.Button("New Thread")

        # Event handlers
        def clear_chat():
            interface.original_file_path = None
            interface.display_file_path = None
            return [], None

        def new_thread():
            interface.current_thread_id = str(time.time())
            # Don't clear file paths on new thread - keep current image
            return [], interface.display_file_path

        def handle_file_upload(file):
            return interface.handle_upload(file.name)

        txt.submit(
            interface.process_message,
            inputs=[txt, image_display, chatbot],
            outputs=[chatbot, image_display]
        ).then(
            lambda: "",
            None,
            [txt]
        )

        upload_button.upload(
            handle_file_upload,
            inputs=upload_button,
            outputs=image_display
        )
        
        dicom_upload.upload(
            handle_file_upload,
            inputs=dicom_upload,
            outputs=image_display
        )

        clear_btn.click(clear_chat, outputs=[chatbot, image_display])
        new_thread_btn.click(new_thread, outputs=[chatbot, image_display])

    return demo

demo = create_demo(agent)
demo.launch(share=True)

* Running on local URL:  http://127.0.0.1:7870
* Running on public URL: https://064c1a7bfd9a020f96.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Executing tool: {'name': 'chest_xray_segmentation', 'args': {'image_path': 'temp/upload_1732291692.dcm'}, 'id': 'call_OTgIctkVdSaMhOJxH81PfuwB', 'type': 'tool_call'}
Returning to model processing!
